In [1]:
%pip install opencv-python mediapipe pandas scikit-learn tensorflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import cv2
import mediapipe as mp
import pandas as pd

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
kategori = ['tangkisan dalam', 'tangkisan luar']  # sesuaikan dengan folder
data_all = []

for label in kategori:
    folder = f'dataset/tangkisan/{label}'
    for filename in os.listdir(folder):
        if filename.endswith('.mp4'):
            cap = cv2.VideoCapture(os.path.join(folder, filename))
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(image)
                if results.pose_landmarks:
                    row = []
                    for lm in results.pose_landmarks.landmark:
                        row.extend([lm.x, lm.y, lm.z, lm.visibility])
                    row.append(label)
                    data_all.append(row)
            cap.release()

pose.close()

# Simpan ke DataFrame
num_landmarks = 33 * 4  # x, y, z, v
columns = [f'{name}_{axis}' for name in range(33) for axis in ['x', 'y', 'z', 'v']]
columns.append('label')

df = pd.DataFrame(data_all, columns=columns)
df.to_csv('csv/pose_dataset_tangkisan2.csv', index=False)
print("✅ Dataset disimpan ke pose_dataset_tangkisan2.csv")


✅ Dataset disimpan ke pose_dataset_tangkisan2.csv


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ✅ 1. Baca dataset
df = pd.read_csv('csv/pose_dataset_tangkisan2.csv')

# ✅ 2. Ambil hanya titik-titik tangan (6 keypoint × 4 = 24 fitur)
selected_indices = [11, 12, 13, 14, 15, 16]
selected_columns = []

for i in selected_indices:
    for axis in ['x', 'y', 'z', 'v']:
        selected_columns.append(f"{i}_{axis}")

X = df[selected_columns].values.astype('float32')
y = LabelEncoder().fit_transform(df['label'])
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# ✅ 3. Buat dan latih model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(24,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stop])

model.save("model/model_tangkisan2.h5")
print("✅ Model disimpan ke model_tangkisan2.h5")


Epoch 1/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5625 - loss: 0.7672 - val_accuracy: 0.6329 - val_loss: 0.6527
Epoch 2/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6316 - loss: 0.6548 - val_accuracy: 0.6350 - val_loss: 0.6344
Epoch 3/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6347 - loss: 0.6435 - val_accuracy: 0.6664 - val_loss: 0.6126
Epoch 4/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6478 - loss: 0.6292 - val_accuracy: 0.6798 - val_loss: 0.6064
Epoch 5/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6578 - loss: 0.6223 - val_accuracy: 0.6710 - val_loss: 0.6047
Epoch 6/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6587 - loss: 0.6221 - val_accuracy: 0.6789 - val_loss: 0.5919
Epoch 7/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6561 - loss: 0.6147 - val_accuracy: 0.6986 - val_loss: 0.5863
Epoch 8/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6625 - loss: 0.6018 - val_accuracy: 0.

✅ Model disimpan ke model_tangkisan2.h5


In [ ]:
import tensorflow as tf
from tensorflow import keras

# Load model dari .h5
model = keras.models.load_model("model/model_tangkisan2.h5")

# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan ke file
with open("model/model_tangkisan2.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model TFLite disimpan ke model_tangkisan2.tflite")


INFO:tensorflow:Assets written to: C:\Users\filam\AppData\Local\Temp\tmp5uu70qy9\assets


INFO:tensorflow:Assets written to: C:\Users\filam\AppData\Local\Temp\tmp5uu70qy9\assets


Saved artifact at 'C:\Users\filam\AppData\Local\Temp\tmp5uu70qy9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  1678140743056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1678140744400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1678140744976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1678140744016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1678140743824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1678140745168: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Model berhasil dikonversi ke TFLite
